In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.optim import lr_scheduler

import torchvision
import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms

import random as rand
from random import *
import os
import cv2
import copy
import time
from PIL import Image

In [34]:
train_transforms = {
    'train': transforms.Compose([
        transforms.Resize((400, 400)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize((400, 400)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
}

test_transforms = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [35]:
data_dir = "../../Data_Set/Labeld_Crop_Data/"
trDsets = {x: dset.ImageFolder(os.path.join(data_dir, x), train_transforms[x]) for x in ['train', 'val']}
trLoaders = {x: torch.utils.data.DataLoader(trDsets[x], batch_size=64, shuffle=True, num_workers=4) for x in ['train', 'val']}

teDsets = dset.ImageFolder(os.path.join(data_dir, 'test'), transform=test_transforms)
teLoaders = torch.utils.data.DataLoader(teDsets, batch_size=64, shuffle=False, num_workers=4)

In [36]:
trDsets_sizes = {x: len(trDsets[x]) for x in ['train', 'val']}
class_names = trDsets['train'].classes

In [37]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [38]:
model = models.squeezenet1_1(pretrained=False, num_classes=22)
model = model.to(device)
print(model)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [45]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, factor=0.1, patience=11)

In [46]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=8, device=device):
    
    global_info = []
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    early_stopping = EarlyStopping(patience=11, verbose=True)
    for epoch in range(num_epochs):
        local_info = []
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
                if epoch > 0:
                    scheduler.step(val_loss)
                    
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in trLoaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / trDsets_sizes[phase]
            if phase == 'val':
                val_loss = running_loss / trDsets_sizes['val']
            epoch_acc = running_corrects.double() / trDsets_sizes[phase]

            if phase == 'train':
                local_info.append(epoch_loss)
                ea = epoch_acc.cpu().numpy()
                local_info.append(ea)
            else:
                local_info.append(epoch_loss)
                ea = epoch_acc.cpu().numpy()
                local_info.append(ea)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        lr_get = get_lr(optimizer)
        print("Current learning rate : {:.8f}".format(lr_get))
        global_info.append(local_info)
        
        if phase =='val':
            early_stopping(epoch_loss, model)

            if early_stopping.early_stop:
                print("Early stopping")
                break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
        
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [47]:
torch.cuda.empty_cache()
model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100, device=device)

Epoch 0/99
----------
train Loss: 0.8975 Acc: 0.6876
val Loss: 0.9240 Acc: 0.6584
Current learning rate : 0.00100000
Validation loss decreased (inf --> 0.923968).  Saving model ...
Epoch 1/99
----------
train Loss: 0.7195 Acc: 0.7469
val Loss: 0.5142 Acc: 0.8262
Current learning rate : 0.00100000
Validation loss decreased (0.923968 --> 0.514231).  Saving model ...
Epoch 2/99
----------
train Loss: 0.5295 Acc: 0.8163
val Loss: 0.4834 Acc: 0.8330
Current learning rate : 0.00100000
Validation loss decreased (0.514231 --> 0.483361).  Saving model ...
Epoch 3/99
----------
train Loss: 0.4304 Acc: 0.8489
val Loss: 0.5989 Acc: 0.7946
Current learning rate : 0.00100000
EarlyStopping counter: 1 out of 11
Epoch 4/99
----------
train Loss: 0.4257 Acc: 0.8506
val Loss: 0.4075 Acc: 0.8538
Current learning rate : 0.00100000
Validation loss decreased (0.483361 --> 0.407457).  Saving model ...
Epoch 5/99
----------
train Loss: 0.3664 Acc: 0.8700
val Loss: 0.4464 Acc: 0.8466
Current learning rate : 0.0

val Loss: 0.1233 Acc: 0.9712
Current learning rate : 0.00100000
EarlyStopping counter: 5 out of 11
Epoch 51/99
----------
train Loss: 0.0832 Acc: 0.9739
val Loss: 0.5237 Acc: 0.8801
Current learning rate : 0.00100000
EarlyStopping counter: 6 out of 11
Epoch 52/99
----------
train Loss: 0.1047 Acc: 0.9706
val Loss: 0.1238 Acc: 0.9724
Current learning rate : 0.00100000
EarlyStopping counter: 7 out of 11
Epoch 53/99
----------
train Loss: 0.0999 Acc: 0.9707
val Loss: 0.1539 Acc: 0.9624
Current learning rate : 0.00100000
EarlyStopping counter: 8 out of 11
Epoch 54/99
----------
train Loss: 0.0830 Acc: 0.9749
val Loss: 0.1191 Acc: 0.9760
Current learning rate : 0.00100000
EarlyStopping counter: 9 out of 11
Epoch 55/99
----------
train Loss: 0.0584 Acc: 0.9824
val Loss: 0.1218 Acc: 0.9752
Current learning rate : 0.00100000
EarlyStopping counter: 10 out of 11
Epoch 56/99
----------
train Loss: 0.0528 Acc: 0.9849
val Loss: 0.0942 Acc: 0.9824
Current learning rate : 0.00100000
Validation loss d

In [48]:
torch.save(model_ft, 'lotte_model_squeezenet.pt')

In [49]:
testModel = torch.load('lotte_model_squeezenet.pt', map_location=device)

In [50]:
correct = 0
total = 0
with torch.no_grad():
    for data in teLoaders:
        images, labels = data
        images, labels = Variable(images.float().cuda()), Variable(labels.float().cuda())
        
        outputs = testModel(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of test images: %.2f %%' % (100 * correct / total))

Accuracy of test images: 98.35 %


In [51]:
tag_classes = ['ID_gum', 'buttering', 'couque_coffee', 'chocopie', 'cidar', 'couque_white', 'coke', 'diget_ori', 'diget_choco', 'gumi_gumi', 'homerunball', 'jjolbyung_noodle', 'juicyfresh', 'jjolbyung_ori', 'spearmint', 'squid_peanut', 'samdasu', 'tuna', 'toreta', 'vita500', 'welchs', 'zec']
tag_dict = dict()
for i, label in enumerate(tag_classes):
    tag_dict[i] = label

print(tag_dict)

{0: 'ID_gum', 1: 'buttering', 2: 'couque_coffee', 3: 'chocopie', 4: 'cidar', 5: 'couque_white', 6: 'coke', 7: 'diget_ori', 8: 'diget_choco', 9: 'gumi_gumi', 10: 'homerunball', 11: 'jjolbyung_noodle', 12: 'juicyfresh', 13: 'jjolbyung_ori', 14: 'spearmint', 15: 'squid_peanut', 16: 'samdasu', 17: 'tuna', 18: 'toreta', 19: 'vita500', 20: 'welchs', 21: 'zec'}


In [52]:
class_correct = list(0 for i in range(len(class_names)))
class_total = list(0 for i in range(len(class_names)))
with torch.no_grad():
    for data in teLoaders:
        images, labels = data
        images, labels = Variable(images.float().cuda()), Variable(labels.float().cuda())
        
        outputs = model_ft(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()

        for i in range(c.size(0)):
            label = labels[i]
            class_correct[int(label.item())] += c[i].item()
            class_total[int(label.item())] += 1

for i in range(len(class_names)):
    print('Accuracy of %5s : %.2f %%' % (tag_dict[int(class_names[i])], 100 * class_correct[i] / class_total[i]))

Accuracy of ID_gum : 97.42 %
Accuracy of buttering : 100.00 %
Accuracy of homerunball : 99.57 %
Accuracy of jjolbyung_noodle : 100.00 %
Accuracy of juicyfresh : 100.00 %
Accuracy of jjolbyung_ori : 99.58 %
Accuracy of spearmint : 95.40 %
Accuracy of squid_peanut : 100.00 %
Accuracy of samdasu : 99.51 %
Accuracy of  tuna : 97.65 %
Accuracy of toreta : 100.00 %
Accuracy of vita500 : 98.95 %
Accuracy of couque_coffee : 99.57 %
Accuracy of welchs : 99.19 %
Accuracy of   zec : 98.99 %
Accuracy of chocopie : 98.95 %
Accuracy of cidar : 98.81 %
Accuracy of couque_white : 97.98 %
Accuracy of  coke : 97.67 %
Accuracy of diget_ori : 90.98 %
Accuracy of diget_choco : 95.47 %
Accuracy of gumi_gumi : 100.00 %
